# 🏭 ImgCraft V2 Ultimate - Colab Runner
## Hướng dẫn sử dụng:
1. Upload thư mục `imgcraft_pipeline_v2` lên Google Drive.
2. Kết nối Runtime với GPU (L4, A100 khuyến nghị).
3. Chạy lần lượt các Cell bên dưới.
4. **Lưu ý:** Tunnel Cloudflare sẽ không bị giới hạn 60 phút như Pinggy.

In [ ]:
# 1. Kết nối Google Drive
from google.colab import drive
import os

drive.mount('/content/drive')

def find_project_path(search_root="/content/drive/MyDrive", target_file="pipeline_v2.py", max_depth=3):
    print(f"🔍 Đang tìm kiếm '{target_file}' trong {search_root}...")
    for root, dirs, files in os.walk(search_root):
        if target_file in files:
            return root
        current_depth = root[len(search_root):].count(os.sep)
        if current_depth >= max_depth:
            del dirs[:] 
    return None

PROJECT_PATH = find_project_path()

if PROJECT_PATH:
    os.chdir(PROJECT_PATH)
    print(f"✅ PROJECT ROOT: {PROJECT_PATH}")
else:
    print("❌ Project not found. Please upload 'imgcraft_pipeline_v2'.")

In [ ]:
# 2. Cài đặt Môi trường (FP8 Support)
import os
import subprocess

def run(cmd): 
    print(f"🖥️ Running: {cmd}")
    subprocess.run(cmd, shell=True, check=True)

# --- ENABLE SWAP (PREVENT RAM CRASH) ---
try:
    print("💾 Creating 16GB Swap File to prevent OOM...")
    run("fallocate -l 16G /swapfile")
    run("chmod 600 /swapfile")
    run("mkswap /swapfile")
    run("swapon /swapfile")
    print("✅ Swap Enabled.")
except Exception as e:
    print(f"⚠️ Failed to enable Swap: {e}")

print("📦 System Dependencies...")
run("apt-get install -y aria2 libgl1-mesa-glx")

os.chdir('/content')
if not os.path.exists("/content/ComfyUI"):
    run("git clone https://github.com/jofix2004/ComfyUI.git")

print("📥 Python Requirements...")
reqs = [
    "torch", "torchvision", "torchaudio", "xformers", "diffusers", "transformers", 
    "accelerate", "einops", "spandrel", "opencv-python", "Pillow", "numpy", "gradio", 
    "huggingface_hub", "aria2", "torchsde", "av", "scipy", "sentencepiece", 
    "safetensors", "omegaconf", "comfy-cli", "fastapi", "uvicorn", "python-multipart"
]
run(f"pip install {' '.join(reqs)}")

print("🌐 Installing Cloudflare Tunnel (cloudflared)...")
run("wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb")
run("dpkg -i cloudflared-linux-amd64.deb")

print("⬇️ Models...")
models = {
    "unet": [("flux1-kontext-dev-fp8-e4m3fn.safetensors", "https://huggingface.co/6chan/flux1-kontext-dev-fp8/resolve/main/flux1-kontext-dev-fp8-e4m3fn.safetensors")],
    "vae": [("ae.sft", "https://huggingface.co/Isi99999/Upscalers/resolve/main/Flux/ae.sft")],
    "clip": [("clip_l.safetensors", "https://huggingface.co/Isi99999/Upscalers/resolve/main/Flux/clip_l.safetensors"), 
             ("t5xxl_fp8_e4m3fn.safetensors", "https://huggingface.co/Isi99999/Upscalers/resolve/main/Flux/t5xxl_fp8_e4m3fn.safetensors")],
    "loras": [("flux_1_turbo_alpha.safetensors", "https://huggingface.co/Isi99999/Upscalers/resolve/main/Flux/flux_1_turbo_alpha.safetensors"),
              ("AniGaKontext2080v3_Full_000000650.safetensors", "https://huggingface.co/TranLinh2004/AniGaKontext2080v3_Full/resolve/main/AniGaKontext2080v3_Full_000000650.safetensors")]
}

for folder, items in models.items():
    dest = f"/content/ComfyUI/models/{folder}"
    os.makedirs(dest, exist_ok=True)
    for fname, url in items:
        if not os.path.exists(f"{dest}/{fname}"):
            run(f"aria2c -c -x 16 -s 16 -k 1M -d {dest} -o {fname} {url}")

if 'PROJECT_PATH' in locals(): os.chdir(PROJECT_PATH)
print("✅ Environment Ready!")

In [ ]:
# 3. Khởi động Ultimate Dashboard (CLOUDFLARE TUNNEL)
import threading
import time
import subprocess
import sys
import os
import re
import pipeline_v2

print("📦 Setting up Server...")
# Workers
sys.path.append("/content/ComfyUI")
sys.path.append("/content/ComfyUI/custom_nodes")

# Check RAM & Swap
print("💾 Checking Memory (RAM + Swap)...")
subprocess.run("free -h", shell=True)

pipeline_v2.ensure_dirs()

def start_workers():
    print("🏭 Starting Backend Processing Workers...")
    t1 = threading.Thread(target=pipeline_v2.worker_crop, daemon=True)
    t2 = threading.Thread(target=pipeline_v2.worker_clean, daemon=True)
    t3 = threading.Thread(target=pipeline_v2.worker_stitch, daemon=True)
    t1.start(); t2.start(); t3.start()
    print("✅ Workers active.")

start_workers()

# Server
print("🚀 Starting ImgCraft V2 Ultimate Server (Port 8000)...")
subprocess.run("pkill -f uvicorn", shell=True)
subprocess.run("pkill -f cloudflared", shell=True)

subprocess.Popen(["uvicorn", "server:app", "--host", "0.0.0.0", "--port", "8000"], 
                  cwd=os.getcwd(), stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
time.sleep(5)

# Tunnel
print("🌐 Generating Public Link (Cloudflare Tunnel)...")
print("👇👇👇 ACCESS DASHBOARD HERE 👇👇👇")
print("="*50)

# Start cloudflared in background and extract the URL
cmd = "cloudflared tunnel --url http://localhost:8000"
process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

logged_urls = set()

try:
    for line in iter(process.stdout.readline, ''):
        if "trycloudflare.com" in line:
            # Robust URL extraction using Regex
            match = re.search(r"https://[a-zA-Z0-9-]+\\.trycloudflare\\.com", line)
            if match:
                url = match.group(0)
                if url not in logged_urls:
                    print(f"\n🔗 URL: {url}\n")
                    print("="*50)
                    logged_urls.add(url)
            else:
                # Fallback: if regex fails, manually search for characters
                if "https://" in line:
                    url_part = line.split("https://")[1].split(" ")[0].strip()
                    full_url = f"https://{url_part}"
                    if full_url not in logged_urls:
                        print(f"\n🔗 URL: {full_url}\n")
                        print("="*50)
                        logged_urls.add(full_url)
        # Optional: uncomment to see all logs
        # print(line.strip())
except KeyboardInterrupt:
    process.kill()
    print("\n🛑 Tunnel stopped.")
